In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd /content/drive/MyDrive/TalkLikeMom/src/Classifier

In [1]:
import sys
sys.path.append('.')

In [2]:
# !pip install -q emoji pythainlp

In [3]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

# from transformers import AutoTokenizer
# import transformers
import torch
import numpy as np
import scipy as sp
# import shap

from utils import dump_jsonl, load_jsonl, set_random_seed
# import torch
# from transformers import AutoModelForSequenceClassification, AdamW, BertConfig

In [4]:
import pickle
def load_obj_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

In [5]:
# !pip install -q pythainlp emoji==1.7

In [6]:
import emoji
import pythainlp
from emoji import UNICODE_EMOJI

def is_emoji(s):
    for char in s:
        if char in UNICODE_EMOJI["en"]:
            return True
    return False

labelmap = {
    '1. Close': "1. Close",
    '2. Know each other': "2. Acquainted",
    "3. Don't know each other": "3. Unfamiliar",


    '0. Very respect': "0. Highly Respectful",
    '1. Respect': "1. Respectful",
    '2. Normal': "2. Normal",
    '3. Not respect': "3. Disrespectful"
}

def label2newlabel(label):
    if type(label) is list or type(label) is np.ndarray:
        return [labelmap[l] if l in labelmap else l for l in label ]

    if label in labelmap:
        return labelmap[label]
    return label




In [7]:
fn_abs = False
suffix = "abs" if fn_abs else ''

lexicon_paths = [
  f"Lexicons/v3/lexicon_task1_clse{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task2_clse{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task3_clse{suffix}.jsonl",

  f"Lexicons/v3/lexicon_task1_auth{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task2_auth{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task3_auth{suffix}.jsonl"
]

def print_lexicon(lexicons, lexicons_by_feat):
    s = ""
    for label in sorted(lexicons.keys()):
        if label=="overall":
          continue

        for lextype in ["all", "pronoun", "particles", "misspelling"]:

          lexlabel = {
              "all": "All",
              "pronoun": "Pronoun",
              "particles": "Sentence-final Particle",
              "misspelling": "Spelling Variation",
          }




          if lextype=="all":
            s += "            "+label2newlabel(label) +f" & {lexlabel[lextype]} & "
            words = lexicons[label]
          else:
            s += "            " +f" & {lexlabel[lextype]} & "
            words = lexicons_by_feat[label][lextype]

          sorted_words = dict(sorted(words.items(), key=lambda item: -item[1]))
          for k in list(sorted_words.keys())[0:10]:
              unique = False

              if pythainlp.util.isthai(k):
                  s += "\\thaitext{"+k+"}, "
              elif is_emoji(k):
                  kk = ""
                  for ch in k:
                      if ch in UNICODE_EMOJI["en"]:
                        ch = emoji.demojize(ch)
                        ch = ch.replace("_", "-").replace(":", "")
                        ch = "\emoji{"+ch+"}"
                        kk += ch
                      else:
                        kk += ch

                  s += f"{kk}, "
              else:
                  s += f"{k}, "

          s += "\\\\\n"

        s += "            \hline\n"
    # s += "            \hline"
    # print(s)
    return s

texts = []
for path in lexicon_paths:
  lexicons, lexicons_by_feat = load_jsonl(path)
  s = print_lexicon(lexicons, lexicons_by_feat)
  texts.append(s)
  # print(s)
  # break

Loaded 2 records from Lexicons/v3/lexicon_task1_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task2_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task1_auth.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task2_auth.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_auth.jsonl


In [8]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (texts[0], texts[3]),
    (texts[1], texts[4]),
    (texts[2], texts[5]),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):

    printed_text += "\subsection{"+section+"}"

    printed_text += '''
\\begin{longtable}[h]{
        p{\dimexpr 0.20\linewidth-2\\tabcolsep}
        p{\dimexpr 0.25\linewidth-2\\tabcolsep}
        p{\dimexpr 0.55\linewidth-2\\tabcolsep}
    }
        \hline
        Relationship Label  & Lexicon Category & Lexicons\\\\
        \hline
        \endfirsthead
        \hline
        Relationship Label  & Lexicon Category & Lexicons\\\\
        \hline
        \endhead


        \multicolumn{3}{l}{\\textit{Closeness}} \\\\
        \hline
'''
    printed_text += t1
    printed_text += '''
            \hline

        \multicolumn{3}{l}{\\textit{Respect}} \\\\
        \hline
'''

    printed_text += t2
    printed_text += '''
            \hline
\end{longtable}
\clearpage
\n\n
'''

# print(printed_text)

In [9]:
# lexicon_paths = [
#   # "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   # "Lexicons/v3/lexicon_task1_auth.jsonl",
#   # "Lexicons/v3/lexicon_task2_auth.jsonl",
#   # "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# allSelectedWords = set()
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)

#   selectedWords = set()
#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))

#     keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
#     keys = keys[0:200]
#     words = set(keys)
#     selectedWords.update(words)
#   print("Final", len(selectedWords))

#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))
#     for f in lexicons_by_feat[cat]:
#       feats = set(lexicons_by_feat[cat][f].keys())
#       # if f=="pronoun":
#       #   print(sorted(feats))
#       N = len(lexicons_by_feat[cat][f].keys())
#       print(f, len(feats&selectedWords), N, f"{len(feats&selectedWords)*100/N:.0f}%")

#   allSelectedWords.update(selectedWords)
#   print("==============")

# print(len(allSelectedWords))

In [10]:
# rows = []
# for word in allSelectedWords:
#   rows.append({
#       "คำ (Word)": word,
#       "สนิทกันมาก (Intimate)": "",
#       "สนิทกัน (Close)": "",
#       "แค่คนรู้จักกัน (Acquainted)": "",
#       "ไม่รู้จักกัน (Unfamiliar)": "",
#       "ไม่ชอบหน้ากัน  (Dislike)": "",
#       "บอกไม่ได้ (Cannot describe)": "",
#   })

# import pandas as pd
# df = pd.DataFrame(rows)
# df.to_csv("clse_lexicons.csv", index=False)

In [11]:
# lexicon_paths = [
#   # "Lexicons/v3/lexicon_task1_clse.jsonl",
#   # "Lexicons/v3/lexicon_task2_clse.jsonl",
#   # "Lexicons/v3/lexicon_task3_clse.jsonl",

#   # "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# allSelectedWords = set()
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)

#   selectedWords = set()
#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))

#     keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
#     keys = keys[0:200]
#     words = set(keys)
#     selectedWords.update(words)
#   print("Final", len(selectedWords))

#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))
#     for f in lexicons_by_feat[cat]:
#       feats = set(lexicons_by_feat[cat][f].keys())
#       # if f=="pronoun":
#       #   print(sorted(feats))
#       N = len(lexicons_by_feat[cat][f].keys())
#       print(f, len(feats&selectedWords), N, f"{len(feats&selectedWords)*100/N:.0f}%")

#   allSelectedWords.update(selectedWords)
#   print("==============")

# print(len(allSelectedWords))

In [12]:
# rows = []
# for word in allSelectedWords:
#   rows.append({
#       "คำ (Word)": word,
#       "ให้เกียรติมาก (Highly Respectful)": "",
#       "ให้เกียรติ (Respectful)": "",
#       "ปกติ (Normal)": "",
#       "ไม่ให้เกียรติ (Disrespectful)": "",
#       "ไม่ให้เกียรติมาก (Highly Disrespectful)": "",
#       "บอกไม่ได้ (Cannot describe)": "",
#   })

# import pandas as pd
# df = pd.DataFrame(rows)
# df.to_csv("auth_lexicons.csv", index=False)

In [40]:
from collections import defaultdict

# ordinal_weights
def get_weights(categories):
    weights = defaultdict(dict)
    if len(categories)==3:
        mat = [[1.00, 0.67, 0.00], 
               [0.67, 1.00, 0.67], 
               [0.00, 0.67, 1.00]]
    elif len(categories)==4:
        mat = [[1.00, 0.83, 0.50, 0.00], 
               [0.83, 1.00, 0.83, 0.50], 
               [0.50, 0.83, 1.00, 0.83], 
               [0.00, 0.50, 0.83, 1.00]]
    elif len(categories)==5:
        mat = [[1, 0.9, 0.7, 0.4, 0.0], 
               [0.9, 1, 0.9, 0.7, 0.4], 
               [0.7, 0.9, 1, 0.9, 0.7], 
               [0.4, 0.7, 0.9, 1, 0.9],
               [0.0, 0.4, 0.7, 0.9, 1]]
    else:
        # Lazy to implement in case of len(categories) > 5
        raise Exception("No Implementation")
        
    for i, l in enumerate(categories): 
        for j, k in enumerate(categories): 
            weights[l][k] = mat[i][j]
    return weights

    
def cal_agreement(df1, df2, column, categories, cat_column):
    merged = pd.merge(df1, df2, on=column)
#     print(len(df1), len(df2), len(merged))
#     assert(len(df1)==len(merged))
    merged = merged.dropna()    
    cnt_matrix = defaultdict(dict)
    acc_matrix = defaultdict(dict)
    
    for l in categories: 
        for k in categories: 
            d = merged
            d = d[d[f"{cat_column}_x"]==k]
            d = d[d[f"{cat_column}_y"]==l]
            cnt_matrix[l][k] = len(d)
    
    for l in categories: 
        d = merged
        d = d[d[f"{cat_column}_x"]==l]
        acc_matrix["x"][l] = len(d)
        
        d = merged
        d = d[d[f"{cat_column}_y"]==l]
        acc_matrix["y"][l] = len(d)
    
    weights = get_weights(categories)
    
    N = len(merged)
    Pa = 0
    for l in categories: 
        for k in categories: 
            Pa += weights[l][k]*cnt_matrix[l][k]/N
    
    Pe = 0
    for l in categories: 
        for k in categories: 
            Pe += weights[l][k]*(acc_matrix["x"][l]/N)*(acc_matrix["y"][k]/N)
    
    if Pe==1:
        raise Exception("Divide by zero")
    
    kappa = (Pa-Pe)/(1-Pe)
    return kappa

In [41]:
ls HumanScores/auth_*

HumanScores/auth_arm.csv    HumanScores/auth_film.csv
HumanScores/auth_b.csv      HumanScores/auth_get.csv
HumanScores/auth_bow.csv    HumanScores/auth_sky.csv
HumanScores/auth_ch.csv     HumanScores/auth_stars.csv
HumanScores/auth_dy.csv     HumanScores/auth_top.csv
HumanScores/auth_earth.csv  HumanScores/auth_view.csv
HumanScores/auth_fern.csv   HumanScores/auth_wee.csv


In [122]:
users = ["arm", "b", "bow", "ch", "dy", "earth", "fern", "film", "get", "sky", "stars", "top", "wee"]

In [123]:
# clse_get

In [124]:
import pandas as pd

tasks = ["auth", "clse"]

scores = {"auth": [], "clse": []}
for task in tasks:
    filenames = [f"HumanScores/{task}_{u}.csv" for u in users]
    for fin in filenames:
        try:
            hum = pd.read_csv(fin, skiprows=[0])
        except:
            print(f"SKIP {fin}")
            continue

        words = {}
        columns = hum.columns
        #     print(columns)
        rows = []
        for idx, row in hum.iterrows():
            w = row["คำ (Word)"]

            for col in columns:
                if col == "Unnamed: 7":
                    break

                if col in [
                    "คำ (Word)",
                    "Unnamed: 7",
                    "บอกไม่ได้ \n(Cannot describe)",
                    "บอกไม่ได้ \r\n(Cannot describe)",
                ]:
                    continue

                if pd.isna(row[col]):
                    continue

                label2score = {
                    "ไม่ชอบหน้ากัน  (Dislike)": -2,
                    "ไม่รู้จักกัน (Unfamiliar)": -1,
                    "แค่คนรู้จักกัน (acquainted)": 0,
                    "สนิทกัน (Close)": 1,
                    "สนิทกันมาก (Intimate)": 2,
                    "ให้เกียรติมาก \n(Highly Respectful)": 2,
                    "ให้เกียรติ (Respectful)": 1,
                    "ปกติ \n(Normal)": 0,
                    "ไม่ให้เกียรติ (Disrespectful)": -1,
                    "ไม่ให้เกียรติมาก (Highly Disrespectful)": -2,
                }

                if w in words:
                    words[w].append(col)
                else:
                    words[w] = [col]

                rows.append({"word": w, "label": col, "score": label2score[col]})

        scores[task].append((fin, pd.DataFrame(rows)))
        print("DONE", fin, len(rows))


DONE HumanScores/auth_arm.csv 872
DONE HumanScores/auth_b.csv 898
DONE HumanScores/auth_bow.csv 908
DONE HumanScores/auth_ch.csv 898
DONE HumanScores/auth_dy.csv 908
DONE HumanScores/auth_earth.csv 883
DONE HumanScores/auth_fern.csv 904
DONE HumanScores/auth_film.csv 897
DONE HumanScores/auth_get.csv 903
DONE HumanScores/auth_sky.csv 906
DONE HumanScores/auth_stars.csv 323
DONE HumanScores/auth_top.csv 903
DONE HumanScores/auth_wee.csv 898
DONE HumanScores/clse_arm.csv 479
DONE HumanScores/clse_b.csv 450
DONE HumanScores/clse_bow.csv 447
DONE HumanScores/clse_ch.csv 388
DONE HumanScores/clse_dy.csv 249
DONE HumanScores/clse_earth.csv 399
DONE HumanScores/clse_fern.csv 543
DONE HumanScores/clse_film.csv 523
SKIP HumanScores/clse_get.csv
DONE HumanScores/clse_sky.csv 397
DONE HumanScores/clse_stars.csv 245
DONE HumanScores/clse_top.csv 551
DONE HumanScores/clse_wee.csv 450


In [125]:
CLOSENESS_LABELS = [
    'ไม่ชอบหน้ากัน  (Dislike)',
    'ไม่รู้จักกัน (Unfamiliar)',
    'แค่คนรู้จักกัน (acquainted)',
    'สนิทกัน (Close)',
    'สนิทกันมาก (Intimate)',
]

AUTHORITY_LABELS = [
    'ให้เกียรติมาก \n(Highly Respectful)',
    'ให้เกียรติ (Respectful)',
    'ปกติ \n(Normal)',
    'ไม่ให้เกียรติ (Disrespectful)',
    'ไม่ให้เกียรติมาก (Highly Disrespectful)',
]

for task in tasks:
    if task=="clse":
        LABELS = CLOSENESS_LABELS
    else:
        LABELS = AUTHORITY_LABELS

    print("LEN", len(scores[task]))

    for i in range(len(scores[task])):
        fin1, data1 = scores[task][i]
        all_iaa = []
        for j in range(len(scores[task])):
            if i==j:
                continue
                
            _, data2 = scores[task][j]
            iaa = cal_agreement(data1, data2, column="word", categories=LABELS, cat_column="label")
            M = pd.merge(data1, data2, on="word")
            print(i, j, iaa, len(data1), len(data2), len(M))
            all_iaa.append(iaa)

    
        avg_iaa = sum(all_iaa)/len(all_iaa)
        print(fin1, "AVG", avg_iaa)
        print("============")
    

LEN 13
0 1 0.6192713262637028 872 898 863
0 2 0.63598819659246 872 908 870
0 3 0.6313825250072483 872 898 866
0 4 0.6342332809787535 872 908 871
0 5 0.7535291738050593 872 883 864
0 6 0.577342664492725 872 904 867
0 7 0.5972578215400749 872 897 862
0 8 0.5913622014476444 872 903 866
0 9 0.6414542175302737 872 906 869
0 10 0.7241841096292495 872 323 310
0 11 0.5956798213125805 872 903 868
0 12 0.6311976431921142 872 898 866
HumanScores/auth_arm.csv AVG 0.6360735818159905
1 0 0.6192713262637023 898 872 863
1 2 0.6429634956933113 898 908 895
1 3 0.661818773750652 898 898 886
1 4 0.6473176324725831 898 908 896
1 5 0.663849821605874 898 883 874
1 6 0.568020525932211 898 904 890
1 7 0.9603588121587129 898 897 899
1 8 0.5966923548383851 898 903 892
1 9 0.6398421075213465 898 906 893
1 10 0.6894301126085746 898 323 315
1 11 0.8278562995364465 898 903 895
1 12 0.6552051413780987 898 898 886
HumanScores/auth_b.csv AVG 0.6810522003133248
2 0 0.63598819659246 908 872 870
2 1 0.6429634956933092 908

4 3 0.607919658653346 249 388 220
4 5 0.648638847553427 249 399 223
4 6 0.5434078206432265 249 543 236
4 7 0.6253251554628777 249 523 234
4 8 0.6381737876922875 249 397 219
4 9 0.6801786236410515 249 245 188
4 10 0.5410035104389366 249 551 235
4 11 0.5973832177349078 249 450 231
HumanScores/clse_dy.csv AVG 0.5940271921146589
5 0 0.5872095178088289 399 479 330
5 1 0.5402253194481373 399 450 324
5 2 0.5741644278526978 399 447 346
5 3 0.8940194420963653 399 388 374
5 4 0.648638847553427 399 249 223
5 6 0.44995980512750944 399 543 360
5 7 0.560036220640678 399 523 353
5 8 0.9520879770731933 399 397 383
5 9 0.7160999961201712 399 245 224
5 10 0.4480456766967597 399 551 359
5 11 0.5924213178136354 399 450 346
HumanScores/clse_earth.csv AVG 0.632991686202855
6 0 0.568146736230751 543 479 415
6 1 0.6478707605050773 543 450 393
6 2 0.6526220514225065 543 447 395
6 3 0.5239851718939509 543 388 352
6 4 0.5434078206432263 543 249 236
6 5 0.44995980512750977 543 399 360
6 7 0.8639448367184628 543 5

In [126]:
from sklearn.metrics import confusion_matrix

In [127]:
# users[9], users[11]

In [55]:
# m = pd.merge(data1, data2, on="word")
# m[m["label_x"]==m["label_y"]]
# confusion_matrix(m["label_x"], m["label_y"], labels=CLOSENESS_LABELS)

In [56]:
# words

In [139]:
lexicon_paths = [
    # "Lexicons/v3/lexicon_task1_clse.jsonl",
    "Lexicons/v3/lexicon_task2_clse.jsonl",
    "Lexicons/v3/lexicon_task3_clse.jsonl",
    # "Lexicons/v3/lexicon_task1_auth.jsonl",
    # "Lexicons/v3/lexicon_task2_auth.jsonl",
    # "Lexicons/v3/lexicon_task3_auth.jsonl"
]

from collections import defaultdict

data = {}
for path in lexicon_paths:
    lexicons, lexicons_by_feat = load_jsonl(path)
    rows = []
    for cat in lexicons:

        keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
        keys = keys[0:200]
        
        
        
        for w in keys:
            o = {
                "word": w,
                "shapley": (lexicons[cat][w]),
                "category": cat,
                "path": path,
            }
            
            for feat in lexicons_by_feat["overall"]:
                if w in lexicons_by_feat["overall"][feat]:
                    o[feat] = True
                else:
                    o[feat] = False
    
            rows.append(o)

    df = pd.DataFrame(rows)
    M = df
    for idx, score in enumerate(scores["clse"]):
        fin, D = score
        M = pd.merge(M, D, how="left", on="word", suffixes=("", f"_{idx}"))
    
    agg_labels = {
        "common": [], 
        "common_w_cutoff": [], 
        "avg": [],
        "avg_w_cutoff": [], 
    }
    
    for idx, row in M.iterrows():
        values = defaultdict(int)
        for col in M.columns:
            if not col.startswith("score_"):
                continue
            
            if pd.isna(row[col]):
                continue
                
            values[row[col]] += 1
        
        
        if values=={}:
            agg_labels["common"].append(None)
            agg_labels["common_w_cutoff"].append(None)
            agg_labels["avg"].append(None)
            agg_labels["avg_w_cutoff"].append(None)
            continue
        
            
        common_label = max(values, key=values.get)
        agg_labels["common"].append(common_label)
        
        if values[common_label] >= len(scores["clse"])*0.5:
            agg_labels["common_w_cutoff"].append(common_label)
        else:
            agg_labels["common_w_cutoff"].append(None)
        
        weight_avg_a = 0
        weight_avg_b = 0
        for w, c in values.items():
            weight_avg_a += w*c
            weight_avg_b += c
        
        weight_avg = weight_avg_a/weight_avg_b
        agg_labels["avg"].append(weight_avg)
        
        if values[common_label] >= len(scores["clse"])*0.5:
            agg_labels["avg_w_cutoff"].append(weight_avg)
        else:
            agg_labels["avg_w_cutoff"].append(None)
            
    for agg_type in agg_labels:
        M[agg_type] = agg_labels[agg_type]
    
#         break
    M = M.rename(columns={"label": "label_0", "score": "score_0"})
    M.to_csv("HumanScores/cls.csv", index=False)
    data[path] = M


Loaded 2 records from Lexicons/v3/lexicon_task2_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_clse.jsonl


In [147]:
# o = [
#     'ไม่ชอบหน้ากัน  (Dislike)',
#     'ไม่รู้จักกัน (Unfamiliar)',
#     'แค่คนรู้จักกัน (Acquainted)',
#     'สนิทกัน (Close)',
#     'สนิทกันมาก (Intimate)',
# ]

# ho = [ '1. Close', '2. Know each other', "3. Don't know each other", "overall"]

from scipy import stats

for path in lexicon_paths:
    M = data[path]
    print(path)
    for idx, score in enumerate(scores["clse"]):
        col = f"score_{idx}"
        d = M[["shapley", col]]
        d = d.dropna()
        res = stats.spearmanr(d["shapley"], d[col])
#         print(d["shapley"], d[col])
        print(f"{col} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
    
    for agg_type in [
        "common",
        "common_w_cutoff",
        "avg",
        "avg_w_cutoff",
    ]:
            
        col = agg_type
        d = M[["shapley", col]]
        d = d.dropna()
        res = stats.spearmanr(d["shapley"], d[col])
        print(f"{col} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
        for feat in ['pronoun', 'particles', 'misspelling']:
            d = M[M[feat]==True][["shapley", col]]
            d = d.dropna()
            res = stats.spearmanr(d["shapley"], d[col])
            print(f"\t >> {feat} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
    
    print("============")

Lexicons/v3/lexicon_task2_clse.jsonl
score_0 544 0.180 0.000
score_1 522 0.171 0.000
score_2 511 0.051 0.250
score_3 456 0.056 0.230
score_4 330 0.107 0.052
score_5 470 0.049 0.287
score_6 576 0.164 0.000
score_7 550 0.147 0.001
score_8 463 0.067 0.147
score_9 323 0.194 0.000
score_10 579 0.154 0.000
score_11 517 0.054 0.217
common 667 0.152 0.000
	 >> pronoun 42 0.341 0.027
	 >> particles 35 0.323 0.058
	 >> misspelling 309 0.261 0.000
common_w_cutoff 368 0.107 0.040
	 >> pronoun 29 0.350 0.063
	 >> particles 23 0.115 0.601
	 >> misspelling 194 0.064 0.376
avg 667 0.149 0.000
	 >> pronoun 42 0.411 0.007
	 >> particles 35 0.322 0.059
	 >> misspelling 309 0.272 0.000
avg_w_cutoff 368 0.109 0.036
	 >> pronoun 29 0.353 0.060
	 >> particles 23 0.225 0.303
	 >> misspelling 194 0.146 0.042
Lexicons/v3/lexicon_task3_clse.jsonl
score_0 428 0.204 0.000
score_1 407 0.237 0.000
score_2 403 0.117 0.019
score_3 372 0.111 0.032
score_4 235 0.103 0.116
score_5 379 0.065 0.204
score_6 486 0.147 0.001


In [149]:
lexicon_paths = [
    # "Lexicons/v3/lexicon_task1_clse.jsonl",
#     "Lexicons/v3/lexicon_task2_clse.jsonl",
#     "Lexicons/v3/lexicon_task3_clse.jsonl",
    # "Lexicons/v3/lexicon_task1_auth.jsonl",
    "Lexicons/v3/lexicon_task2_auth.jsonl",
    "Lexicons/v3/lexicon_task3_auth.jsonl"
]

from collections import defaultdict

data = {}
for path in lexicon_paths:
    lexicons, lexicons_by_feat = load_jsonl(path)
    rows = []
    for cat in lexicons:

        keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
        keys = keys[0:200]

        for w in keys:
            o = {
                "word": w,
                "shapley": (lexicons[cat][w]),
                "category": cat,
                "path": path,
            }
            
            for feat in lexicons_by_feat["overall"]:
                if w in lexicons_by_feat["overall"][feat]:
                    o[feat] = True
                else:
                    o[feat] = False
    
            rows.append(o)

    df = pd.DataFrame(rows)
    M = df
    for idx, score in enumerate(scores["auth"]):
        fin, D = score
        M = pd.merge(M, D, how="left", on="word", suffixes=("", f"_{idx}"))
    
    agg_labels = {
        "common": [], 
        "common_w_cutoff": [], 
        "avg": [],
        "avg_w_cutoff": [], 
    }
    
    for idx, row in M.iterrows():
        values = defaultdict(int)
        for col in M.columns:
            if not col.startswith("score_"):
                continue
            
            if pd.isna(row[col]):
                continue
                
            values[row[col]] += 1
        
        
        if values=={}:
            agg_labels["common"].append(None)
            agg_labels["common_w_cutoff"].append(None)
            agg_labels["avg"].append(None)
            agg_labels["avg_w_cutoff"].append(None)
            continue
        
            
        common_label = max(values, key=values.get)
        agg_labels["common"].append(common_label)
        
        if values[common_label] >= len(scores["clse"])*0.5:
            agg_labels["common_w_cutoff"].append(common_label)
        else:
            agg_labels["common_w_cutoff"].append(None)
        
        weight_avg_a = 0
        weight_avg_b = 0
        for w, c in values.items():
            weight_avg_a += w*c
            weight_avg_b += c
        
        weight_avg = weight_avg_a/weight_avg_b
        agg_labels["avg"].append(weight_avg)
        
        if values[common_label] >= len(scores["clse"])*0.5:
            agg_labels["avg_w_cutoff"].append(weight_avg)
        else:
            agg_labels["avg_w_cutoff"].append(None)
            
    for agg_type in agg_labels:
        M[agg_type] = agg_labels[agg_type]
    
#         break
    M = M.rename(columns={"label": "label_0", "score": "score_0"})
    M.to_csv("HumanScores/auth.csv", index=False)
    data[path] = M


Loaded 2 records from Lexicons/v3/lexicon_task2_auth.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_auth.jsonl


In [150]:
# hum

In [151]:
# o = [
#     'ให้เกียรติมาก \n(Highly Respectful)',
#     'ให้เกียรติ (Respectful)',
#     'ปกติ \n(Normal)',
#     'ไม่ให้เกียรติ (Disrespectful)',
#     'ไม่ให้เกียรติมาก (Highly Disrespectful)',
# ]

# ho = ['1. Respect', '2. Normal', '3. Not respect', "overall"]

from scipy import stats

for path in lexicon_paths:
    M = data[path]
    print(path)
    for idx, score in enumerate(scores["clse"]):
        col = f"score_{idx}"
        d = M[["shapley", col]]
        d = d.dropna()
        res = stats.spearmanr(d["shapley"], d[col])
#         print(d["shapley"], d[col])
        print(f"{col} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
    
    for agg_type in [
        "common",
        "common_w_cutoff",
        "avg",
        "avg_w_cutoff",
    ]:
            
        col = agg_type
        d = M[["shapley", col]]
        d = d.dropna()
        res = stats.spearmanr(d["shapley"], d[col])
        print(f"{col} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
        for feat in ['pronoun', 'particles', 'misspelling']:
            d = M[M[feat]==True][["shapley", col]]
            d = d.dropna()
            res = stats.spearmanr(d["shapley"], d[col])
            print(f"\t >> {feat} {len(d)} {res.statistic:.3f} {res.pvalue:.3f}")
    
    print("============")

Lexicons/v3/lexicon_task2_auth.jsonl
score_0 758 0.074 0.042
score_1 786 0.117 0.001
score_2 793 0.087 0.014
score_3 783 0.051 0.156
score_4 792 0.091 0.011
score_5 771 0.101 0.005
score_6 790 0.068 0.055
score_7 786 0.099 0.006
score_8 785 0.089 0.013
score_9 791 0.075 0.036
score_10 295 -0.072 0.218
score_11 788 0.115 0.001
common 797 0.089 0.012
	 >> pronoun 7 0.118 0.801
	 >> particles 21 -0.032 0.891
	 >> misspelling 231 0.077 0.242
common_w_cutoff 771 0.076 0.034
	 >> pronoun 7 0.118 0.801
	 >> particles 21 -0.032 0.891
	 >> misspelling 227 0.083 0.215
avg 797 0.092 0.009
	 >> pronoun 7 0.000 1.000
	 >> particles 21 -0.032 0.891
	 >> misspelling 231 0.044 0.510
avg_w_cutoff 771 0.078 0.030
	 >> pronoun 7 0.000 1.000
	 >> particles 21 -0.032 0.891
	 >> misspelling 227 0.036 0.593
Lexicons/v3/lexicon_task3_auth.jsonl
score_0 775 0.027 0.451
score_1 790 -0.010 0.784
score_2 801 0.072 0.040
score_3 794 0.067 0.061
score_4 800 0.080 0.023
score_5 777 -0.014 0.689
score_6 793 0.037 0.2

In [285]:
# import matplotlib.pyplot as plt
# print(ho[0])
# # dd = df[df["category"]==ho[0]]
# # dd = df
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [272]:
# dd = df[df["category"]==ho[1]]
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [271]:
# dd = df[df["category"]==ho[2]]
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [ ]:
# from scipy import stats
# import pandas as pd

# def cal_corr(shap_lexicons, weights, label_fn):
#     df = []
#     for label in shap_lexicons:
#         count = defaultdict(int)
#         for w in shap_lexicons[label]:
#             if w not in lexicons:
#                 continue

#             for t in lexicons[w]:
#                 count[t] += 1

#         l = label_fn(label)
#         A = []
#         B = []
#         for feat in count:
#             k = (l, feat)
#             if k in weights and weights[k]!="":
#                 A.append(weights[k])
#                 B.append(count[feat])

#         spm = stats.spearmanr(A, B)
#         df.append({
#             "label": label,
#             "N": len(A),
#             "spearman": spm.statistic,
#             "pvalue": spm.pvalue
#         })

#     return pd.DataFrame(df)

In [ ]:
# shap_lexicons = load_jsonl("Classifier/lexicon_task1_clse.jsonl")[0]
# weights = load_obj_values("linear_weights_task1_clse.pkl")

# cal_corr(shap_lexicons, weights, closeness_to_eng1)

In [ ]:
# lexicon_paths = [
#   "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# texts = []
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)
#   print(path)


#   N = 20
#   count = {}
#   # dict_keys(['pronoun', 'particles', 'misspelling'])
#   for cat in lexicons_by_feat:
#     for idx, w in enumerate(lexicons_by_feat[cat]["pronoun"]):
#       if idx >= N:
#         break

#       if w in count:
#         count[w].append(idx)
#       else:
#         count[w] = [idx]

#   words = {
#       (1, "same"):[],
#       (1, "div"):[],
#       (2, "same"):[],
#       (2, "div"):[],
#       (3, "same"):[],
#       (3, "div"):[],
#   }
#   for w in count:
#     n = len(count[w])

#     if n==1:
#       words[(n, "same")].append(w)
#     elif n>=2:
#       # print(w, count[w])
#       mx = max(count[w])
#       mn = min(count[w])

#       if mx-mn > 5:
#         words[(n, "div")].append((w, count[w]))
#       else:
#         words[(n, "same")].append((w, count[w]))

#   for n in words:
#     print(n, len(words[n]))
#     print(words[n])
#     print()
#   print("------------------")

In [69]:
# !wget -q http://www.arts.chula.ac.th/ling/wp-content/uploads/TH-Sarabun_Chula1.1.zip -O font.zip
# !unzip -qj font.zip TH-Sarabun_Chula1.1/THSarabunChula-Regular.ttf

import matplotlib
matplotlib.font_manager.fontManager.addfont('THSarabunChula-Regular.ttf')
matplotlib.rc('font', family='TH Sarabun Chula')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

def quadrant_chart(x, y, xtick_labels=None, ytick_labels=None, data_labels=None,
 highlight_quadrants=None, ax=None):
    """
    Create the classic four-quadrant chart.
    Args:
        x -- array-like, the x-coordinates to plot
        y -- array-like, the y-coordinates to plot
        xtick_labels -- list, default: None, a two-value list xtick labels
        ytick_labels -- list, default: None, a two-value list of ytick labels
        data_labels -- array-like, default: None, data point annotations
        highlight_quadrants -- list, default: None, list of quadrants to
            emphasize (quadrants are numbered 1-4)
        ax -- matplotlib.axes object, default: None, the user can pass their own
            axes object if desired
    """
    # allow user to specify their own axes
    ax = ax if ax else plt.axes()

    data = pd.DataFrame({'x': x, 'y': y, 'data_labels': data_labels})

    # calculate averages up front to avoid repeated calculations
    y_avg = data['y'].mean()
    x_avg = data['x'].mean()

    # set x limits
    adj_x = max((data['x'].max() - x_avg), (x_avg - data['x'].min())) * 1.1
    lb_x, ub_x = (x_avg - adj_x, x_avg + adj_x)
    ax.set_xlim(lb_x, ub_x)

    # set y limits
    adj_y = max((data['y'].max() - y_avg), (y_avg - data['y'].min())) * 1.1
    lb_y, ub_y = (y_avg - adj_y, y_avg + adj_y)
    ax.set_ylim(lb_y, ub_y)

    # set x tick labels
    if xtick_labels:
        ax.set_xticks([(x_avg - adj_x / 2), (x_avg + adj_x / 2)])
        ax.set_xticklabels(xtick_labels)

    # set y tick labels
    if ytick_labels:
        ax.set_yticks([(y_avg - adj_y / 2), (y_avg + adj_y / 2)])
        ax.set_yticklabels(ytick_labels, rotation='vertical', va='center')

    # determine which points to highlight
    if highlight_quadrants:
        quadrants = []
        for x_val, y_val in zip(x, y):
            q = []
            if (x_val >= x_avg) and (y_val >= y_avg):
                q.append(1)
            if (x_val <= x_avg) and (y_val >= y_avg):
                q.append(2)
            if (x_val <= x_avg) and (y_val <= y_avg):
                q.append(3)
            if (x_val >= x_avg) and (y_val <= y_avg):
                q.append(4)
            quadrants.append(q)
        data['quadrant'] = quadrants

        # boolean mask - True = highlight, False = don't highlight
        highlight = data['quadrant'].apply(lambda q: len(set(
        highlight_quadrants) & set(q)) > 0)

        # plot the non-highlighted points within the conditional block
        ax.scatter(data['x'][~highlight], data['y'][~highlight], alpha=0.5,
        c='lightblue', edgecolor='darkblue', zorder=99)
        data = data[highlight]

    # plot remaining points and quadrant lines
    ax.scatter(x=data['x'], y=data['y'], c='lightblue', edgecolor='darkblue',
    zorder=99)
    ax.axvline(x_avg, c='k', lw=1)
    ax.axhline(y_avg, c='k', lw=1)

    # add data labels
    for ix, row in data.iterrows():
        ax.annotate(row['data_labels'], (row['x'], row['y']), xytext=(2, 5),
        textcoords='offset pixels')

In [70]:
# lexicon_paths = [
#   "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# import numpy as np

# texts = []
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)
#   print(path)


#   N = 50
#   count = {}
#   # dict_keys(['pronoun', 'particles', 'misspelling'])
#   for cat in lexicons_by_feat:
#     for idx, w in enumerate(lexicons_by_feat[cat]["misspelling"]):
#       if idx >= N:
#         break

#       if w in count:
#         count[w].append(idx)
#       else:
#         count[w] = [idx]

#   words = []
#   points = []

#   for w in count:
#     n = len(count[w])

#     if n==1:
#       x = count[w][0]
#       y = 0
#       continue
#     elif n>=2:
#       x = np.mean(count[w])
#       y = np.std(count[w])
#     points.append((x, y, w))



#   # plt.rcParams.update({'font.size': 25})
#   x = [p1 for p1, p2, _ in points]
#   y = [p2 for p1, p2, _ in points]
#   l = [p3 for p1, p2, p3 in points]

#   quadrant_chart(
#       x=x,
#       y=y,
#       xtick_labels=['Top', 'Bottom'],
#       ytick_labels=['Low', 'High'],
#       data_labels=l,
#       # highlight_quadrants=[2]
#   )
#   plt.title('', fontsize=16)
#   plt.ylabel('Variance', fontsize=14)
#   plt.xlabel('Average Rank', fontsize=14)
#   plt.show()
#   plt.clf()
#   # break